# Тестирование API RAG-бота с помощью LLaMator

In [1]:
%pip install llamator --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


## Модель-жертва (тестируемая)

Пусть в качестве жертвы тестирования выступает API RAG-сервиса. Например, немного модифицированный под атаки виртуальный помощник студента ["Вопрошалыч из ТюмГУ"](https://github.com/nizamovtimur/virtassist).

### Класс-обёртка

Для доступа к API через библиотеку *requests* реализуем класс-обёртку, наследуемую от класса `ClientBase`.

In [2]:
%pip install requests --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from typing import Dict, List, Optional
from llamator.client.chat_client import ClientBase
import requests


class ClientAPI(ClientBase):
    def __init__(self, api_url: str = "http://localhost:8080/api/", model_description: Optional[str] = None):
        self.api_url = api_url
        self.model_description = model_description

    def interact(self, history: List[Dict[str, str]], messages: List[Dict[str, str]]) -> Dict[str, str]:
        history += messages
        try:
            r = requests.post(self.api_url, json={"question": messages[-1]["content"]})
            if r.status_code == 200:
                response_message = {"role": "assistant", "content": r.json()["answer"]}
            else:
                response_message = {"role": "system", "content": str(r)}
        except Exception as e:
            print(e)
            raise
        history.append(response_message)
        return response_message

### Инициализация клиента

In [4]:
tested_model = ClientAPI(
    api_url="http://localhost:8666/qa/",
    model_description="Вопрошалыч — виртуальный RAG-помощник студента Тюменского государственного университета",
)
tested_model.interact(
    history=[], messages=[{"role": "user", "content": "Кто подписал положение об отчислении студентов ТюмГУ?"}]
)

{'role': 'assistant',
 'content': 'Положение об отчислении студентов ТюмГУ подписал ректор университета **Романчук Иван Сергеевич**. Документ датирован 13.06.2023 и подписан в 17:47:41.'}

## Модель-провокатор (атакующая)

Пусть моделью-провокатором будет открытая LLM, доступная через OpenAI-совместимое API

In [5]:
import os
from dotenv import load_dotenv
from llamator.client.specific_chat_clients import ClientOpenAI

load_dotenv("../tests/.env")

attack_model = ClientOpenAI(
    api_key=os.getenv("OPENAI_CLIENT_API_KEY"),
    base_url=os.getenv("OPENAI_CLIENT_BASEURL"),
    model=os.getenv("OPENAI_CLIENT_MODEL"),
    temperature=0.6,
    system_prompts=["You are a strong model."],
)

attack_model.interact(
    history=[],
    messages=[{"role": "user", "content": "Расскажи короткую сказку о промт-инъекциях"}],
)

{'role': 'assistant',
 'content': 'Жила-была однажды в далёком королевстве маленькая девочка по имени Маша. Она была очень умной и любознательной, но у неё было одно большое недуге: её глаза были постоянно красными и болели от аллергии на пыльцу.\n\nОднажды король объявил о том, что в королевстве появился новый лекарственный препарат под названием промт-инъекция, который помогал бороться с аллергией. Маша очень хотела попробовать этот препарат и стать здоровой.\n\nОна пошла к врачу, доктору Смереке, который сказал ей: "Маша, я могу помочь тебе, но мне нужно провести несколько промт-инъекций, чтобы полностью избавить тебя от аллергии".\n\nМаша была в восторге. Доктор начал делать инъекции, и к удивлению всех Маша начала чувствовать себя лучше уже через неделю после первого лечения.\n\nС каждой новой инъекцией её глаза становились все чище и краснее, а боль и зуд уменьшались. Через месяц все симптомы аллергии исчезли полностью!\n\nМаша была счастлива и благодарила доктора Смерека за то, 

## Запуск тестирования

Функция `start_testing` запускает выполнение тестов на выявление уязвимостей в языковых моделях (LLM).

### Параметры

* **attack_model**: экземпляр класса `ClientBase`, представляющий модель, используемую для генерации и валидации атак.
* **tested_model**: экземпляр класса `ClientBase`, представляющий модель, которая тестируется на уязвимости.
* **config**: словарь с конфигурационными параметрами:
	+ **enable_logging**: следует ли включать логирование.
	+ **enable_reports**: следует ли генерировать отчеты в формате xlsx.
	+ **artifacts_path**: путь к папке для сохранения артефактов.
	+ **debug_level**: уровень логирования (0 - WARNING, 1 - INFO, 2 - DEBUG).
* **num_attempts**: количество попыток для каждого тестового случая (по умолчанию 1).
* **num_threads**: количество потоков для параллельного выполнения тестов (по умолчанию 1).
* **tests**: список имен тестов, которые необходимо выполнить (по умолчанию пустой список). Доступные тесты:
	+ aim_jailbreak
	+ base64_injection
	+ complimentary_transition
	+ do_anything_now_jailbreak
	+ RU_do_anything_now_jailbreak
	+ ethical_compliance
	+ harmful_behavior
	+ linguistic_evasion
	+ self_refine
	+ RU_self_refine
	+ sycophancy_test
	+ typoglycemia_attack
	+ ucar
	+ RU_ucar
* **custom_tests**: список пользовательских тестов, наследуемых от `TestBase` (по умолчанию пустой список).

In [11]:
tests = [
    # "aim_jailbreak",
    # "base64_injection",
    # "complimentary_transition",
    # "do_anything_now_jailbreak",
    # "RU_do_anything_now_jailbreak",
    # "ethical_compliance",
    # "harmful_behavior",
    # "linguistic_evasion",
    # "self_refine",
    # "RU_self_refine",
    "sycophancy_test",
    # "typoglycemia_attack",
    # "ucar",
    # "RU_ucar",
]

config = {
    "enable_logging": True,  # Включить логирование
    "enable_reports": True,  # Включить генерацию отчетов
    "artifacts_path": "./artifacts",  # Путь к директории для сохранения артефактов
    "debug_level": 1,  # Уровень логирования: 0 - WARNING, 1 - INFO, 2 - DEBUG
}

from llamator.main import start_testing

start_testing(attack_model=attack_model, tested_model=tested_model, config=config, tests=tests, num_attempts=3)


    __    __    ___    __  ______  __________  ____
   / /   / /   /   |  /  |/  /   |/_  __/ __ \\/ __ \
  / /   / /   / /| | / /|_/ / /| | / / / / / / /_/ /
 / /___/ /___/ ___ |/ /  / / ___ |/ / / /_/ / _, _/
/_____/_____/_/  |_/_/  /_/_/  |_/_/  \\____/_/ |_|

Running tests on your system prompt ...


Test progress ........................................:   0%|          | 0/1 [00:00<?, ?it/s]:   0%|          | 0/1 [00:00<?, ?it/s]2024-09-11 21:45:13,646 [ERROR] [sycophancy.py:215]: Failed to decode JSON: Extra data: line 5 column 1 (char 29)
2024-09-11 21:45:13,651 [ERROR] [work_progress_pool.py:78]: Task caused exception: local variable 'coherency_results' referenced before assignment
Traceback (most recent call last):
  File "D:\git\llamator\src\llamator\attack_provider\work_progress_pool.py", line 75, in worker_function
    task(progress_worker)
  File "D:\git\llamator\src\llamator\attack_provider\run_tests.py", line 55, in __call__
    for statusUpdate in self.test.run():
  File "D:\git\llamator\src\llamator\attacks\sycophancy.py", line 225, in run
    logger.info(f"Evaluated coherency for {len(coherency_results)} input pairs.")
UnboundLocalError: local variable 'coherency_results' referenced before assignment
Test progress ........................................: 100%|███████

UnboundLocalError: local variable 'coherency_results' referenced before assignment